**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4029 - Ciencia y analítica de datos**

Tecnológico de Monterrey

Prof Grettel Barceló Alonso

**Semana 6**
Ingeniería de características (FE)

---

*   NOMBRE: ERICK ALAN CUELLAR QUINTANILLA
*   MATRÍCULA: A01383577

*   NOMBRE: Fernando Alfonso Garcia
*   MATRÍCULA: A01630011

*   NOMBRE: Luis Pablo Perez Perez
*   MATRÍCULA: A01420939

*   NOMBRE: Rodolfo Antonio Vargas Díaz
*   MATRÍCULA: A01411361

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# DIR = "/content/drive/MyDrive/ColabNotebooks/DataScience/Semana 6"
# os.chdir(DIR)

Mounted at /content/drive


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ColabNotebooks/DataScience/Semana 6'

In [ ]:
import os
DIR = r"C:\Coding\Masters\DataScience\Semana 6\Actividad 6"
os.chdir(DIR)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [ ]:
# https://www.kaggle.com/datasets/austinreese/craigslist-carstrucks-dataLinks to an external site.
cars_df = pd.read_csv('vehicles.csv')
cars_df

In [ ]:
cars_df.info()

# **Parte 1**. Preprocesamiento

1a)    Elimina las columnas que no serán relevantes para esta actividad: `id`, `VIN`, `url`, `image_url`, `description`, `posting_date`, `lat`, `long`.


In [ ]:
filtered_cars_df = cars_df.drop(columns=['id', 'VIN', 'url','image_url', 'description', 'posting_date', 'lat', 'long'])
print(filtered_cars_df.columns)

1b) Almacena en una variable la cantidad de observaciones o registros del dataframe y en otra el número de columnas (después del borrado) para análisis posteriores.

In [ ]:
df_lenght = filtered_cars_df.shape[0]
df_column_num = filtered_cars_df.shape[1]
print(f"Cantidad de observaciones: {df_lenght}")
print(f"Número de columnas en el dataframe: {df_column_num}")


## Valores faltantes

Cuando la cantidad de valores faltantes en una columna es lo suficientemente grande (aproximadamente más del 25%), estimar e ingresar los valores faltantes no tiene sentido, y eliminar el atributo es mejor que estimar los valores faltantes. https://learning.oreilly.com/library/view/hands-on-data-preprocessing/9781801072137/B17397_11_Final_NM_ePub.xhtml#_idParaDest-188

Sin embargo, esto no es una regla aplicable a todos los escenarios y hay muchas discusiones en torno a ello. https://www.quora.com/How-much-missing-data-is-too-much-1

Para esta actividad  utilizaremos un umbral de 0.70 de la cantidad de registros (aquellas columnas que tengan 70% de valores no nulos). Considera además que estamos asumiendo que dichas variables son MCAR o MAR sin ninguna método gráfico o estadístico.

2a) Determina el porcentaje de valores faltantes por columna y elimina aquellas que posean el 30% o más de valores perdidos. Para ello utiliza el parámetro `thresh` de la función `dropna()` ¿Cuántas columnas se eliminaron?


In [ ]:
print('-'*10, 'Porcentaje de valores faltantes por columna','-'*10)
print(filtered_cars_df.isna().sum() * 100 / df_lenght)

#Dropping columns which have more than 70% of observations missing
filtered_cars_df.dropna(axis=1, thresh=cars_df.shape[0]*0.7, inplace=True)
print('-'*10, 'Columnas restantes','-'*10)
print(filtered_cars_df.columns)
print(f'Se eliminaron {df_column_num - filtered_cars_df.shape[1]} columnas')

Además, cuando tenemos grandes conjuntos de datos, podemos eliminar filas que contienen valores nulos. Aunque, no se recomienda si el porcentaje de valores faltantes en el conjunto de datos es superior al 30%
https://medium.com/analytics-vidhya/how-to-deal-with-missing-values-in-data-set-8e8f70ecf155

2b) Borra los registros que contengan `NaN` en las columnas con menos del 5% de valores faltantes empleando el parámetro subset ¿Qué porcentaje de los registros iniciales se eliminaron?


In [ ]:

filtered_cars_df.dropna(subset=['year', 'manufacturer', 'model', 'fuel', 'odometer', 'title_status', 'transmission'], inplace=True)
perc_of_deleted_values = (df_lenght - filtered_cars_df.shape[0]) * 100 / df_lenght
print(f"Porcentaje de registros eliminados: {perc_of_deleted_values:.2f}%")

2c) Si hay variables que aún poseen valores faltantes, utiliza el método de imputación apropiado (`mean, median, mode`)

In [ ]:
print('-'*10, 'Porcentaje de valores faltantes por columna','-'*10)
print(filtered_cars_df.isna().sum() * 100 / df_lenght)

In [ ]:
inputed_cars_df = filtered_cars_df.copy()
inputed_cars_df.fillna({'type': inputed_cars_df['type'].mode()[0]}, inplace=True) #Pandas 3.0 warning, had to change the command
print('-'*10, 'Porcentaje de valores faltantes por columna','-'*10)
print(inputed_cars_df.isna().sum() * 100 / df_lenght)

## Valores atípicos

3a) Obtén el histograma de las variables `price`, `year` y `odometer` para observar su distribución.

In [ ]:
selected_columns = ['price', 'year', 'odometer']

# Seaborn
fig, axes = plt.subplots(3,1, figsize=(10,15))
plt.subplots_adjust(wspace=0.3)
axes = axes.ravel()

for col, ax in zip(inputed_cars_df[selected_columns], axes):
  sns.histplot(x=inputed_cars_df[col], ax=ax, bins=20)
  ax.set(title=f'{col}', xlabel=None)



3b) Añade la columna `age`, calculada a partir de `year`, para determinar los años del vehículo y una vez obtenida, elimina la columna `year`. Genera el histograma de la nueva variable.

In [ ]:
inputed_cars_df['age'] = 2025 - inputed_cars_df['year']
inputed_cars_df.drop('year', axis=1, inplace=True)
inputed_cars_df.columns

In [ ]:
plt.figure(figsize=(12,8))
sns.histplot(x=inputed_cars_df['age'], bins=10)
plt.show()

4a) Para visualizar si hay valores atípicos, genera gráficos de box plot para las tres variables (`price, age, odometer`)

In [ ]:
selected_columns = ['price', 'age', 'odometer']

fig, axes = plt.subplots(3,1, figsize=(10,15))
axes = axes.ravel()
for col, ax in zip(inputed_cars_df[selected_columns], axes):
  sns.boxplot(x=inputed_cars_df[col], ax=ax, showmeans=True)
  ax.set(xlabel=f'{col}')


Si el conjunto de outliers es “relativamente grande” (aunque sabemos que deben andar alrededor de un 5% del conjunto total de datos), realizar una investigación particular sobre ellos que permita tomar mejores decisiones.

4b) Calcula, para cada variable, el porcentaje de sus valores atípicos y elimínalos del dataframe ¿Qué porcentaje de los registros iniciales se han eliminado en total hasta ahora?

In [ ]:
# inputed_df_lenght = inputed_cars_df.shape[0]
outliers_filtered_df = inputed_cars_df.copy()

In [ ]:
def eliminar_outliers(df, col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    initial = df.shape[0]
    df = df[(df[col] >= lower) & (df[col] <= upper)]
    final = df.shape[0]
    print(f"{col} - Registros eliminados: {round((initial - final)/initial * 100, 2)}%")
    return df

for col in selected_columns:
    outliers_filtered_df = eliminar_outliers(outliers_filtered_df, col)

In [ ]:
# percentile_25 = outliers_filtered_df["odometer"].quantile(0.25)
# percentile_75 = outliers_filtered_df["odometer"].quantile(0.75)
# iqr = percentile_75 - percentile_25
# upper_limit = percentile_75 + 1.5 * iqr
# lower_limit = percentile_25 - 1.5 * iqr
# IQR_outliers_odometer = outliers_filtered_df[(outliers_filtered_df["odometer"] < lower_limit) | (outliers_filtered_df["odometer"] > upper_limit)]

# print(f"Longitud del dataframe antes de descartar outliers de odometro: {outliers_filtered_df.shape[0]}")

# outliers_filtered_df.drop(IQR_outliers_odometer.index, inplace=True)

# print(f"Longitud del dataframe despues de descartar outliers de odometro: {outliers_filtered_df.shape[0]}")

In [ ]:

# percentile_25 = outliers_filtered_df["price"].quantile(0.25)
# percentile_75 = outliers_filtered_df["price"].quantile(0.75)
# iqr = percentile_75 - percentile_25
# upper_limit = percentile_75 + 1.5 * iqr
# lower_limit = percentile_25 - 1.5 * iqr
# IQR_outliers_price = outliers_filtered_df[(outliers_filtered_df["price"] < lower_limit) | (outliers_filtered_df["price"] > upper_limit)]

# print(f"Longitud del dataframe antes de descartar outliers de precio: {outliers_filtered_df.shape[0]}")

# outliers_filtered_df.drop(IQR_outliers_price.index, inplace=True)

# print(f"Longitud del dataframe despues de descartar outliers de precio: {outliers_filtered_df.shape[0]}")

In [ ]:
# percentile_25 = outliers_filtered_df["age"].quantile(0.25)
# percentile_75 = outliers_filtered_df["age"].quantile(0.75)
# iqr = percentile_75 - percentile_25
# upper_limit = percentile_75 + 1.5 * iqr
# lower_limit = percentile_25 - 1.5 * iqr
# IQR_outliers_age = outliers_filtered_df[(outliers_filtered_df["age"] < lower_limit) | (outliers_filtered_df["age"] > upper_limit)]

# print(f"Longitud del dataframe antes de descartar outliers de edad: {outliers_filtered_df.shape[0]}")

# outliers_filtered_df.drop(IQR_outliers_age.index, inplace=True)

# print(f"Longitud del dataframe despues de descartar outliers de edad: {outliers_filtered_df.shape[0]}")

In [ ]:
# inputed_cars_df.drop(IQR_outliers_price.index, inplace=True)
# inputed_cars_df.drop(IQR_outliers_age.index, inplace=True)
# inputed_cars_df.drop(IQR_outliers_odometer.index, inplace=True)
print(f"Porcentaje de valores atípicos filtrados: {(df_lenght - outliers_filtered_df.shape[0])* 100 / df_lenght}")

4c) Dibuja nuevamente los histogramas para observar claramente el sesgo de las variables tras la eliminación de los outliers.

In [ ]:
# selected_columns = ['price', 'age', 'odometer']

# Seaborn
fig, axes = plt.subplots(3,1, figsize=(10,15))
plt.subplots_adjust(wspace=0.3)
axes = axes.ravel()

for col, ax in zip(outliers_filtered_df[selected_columns], axes):
  sns.histplot(x=outliers_filtered_df[col], ax=ax, bins=20, kde = True)
  ax.set(title=f'{col}', xlabel=None)

# **Parte 2**. Ingeniería de características

## Transformando variables numéricas

La distribución de las variables determinará que método usar y en qué orden. Por ejemplo, en variables sesgadas se debe realizar primero la transformación  y luego el escalamiento, ya que el efecto deseado es "desinclinar" las distribuciones del eje y ese efecto se maximiza cuando aplica en el rango completo en comparación con aplicarlo en variables con una desviación estándar unitaria (por z-score) o variables en el rango [0,1] (por Min-Max).

Aplicar es escalamiento antes comprimiría la mayoría de los datos en un rango estrecho. Entonces se debe iniciar con transformar la característica en Gaussiana y luego escalarla.

5a) Genera gráficos Q-Q de las variables `price, age, odometer` para observar las diferencias entre sus distribuciones y la normal.


In [ ]:
plt.figure(figsize=(15, 6))

# plt.subplot(1, 3, 1)
# stats.probplot(outliers_filtered_df['price'], dist="norm", plot=plt)
# plt.title(f"Q-Q plot of price")

# plt.subplot(1, 3, 2)
# stats.probplot(outliers_filtered_df['age'], dist="norm", plot=plt)
# plt.title(f"Q-Q plot of age")

# plt.subplot(1, 3, 3)
# stats.probplot(outliers_filtered_df['odometer'], dist="norm", plot=plt)
# plt.title(f"Q-Q plot of odometer")

for col in selected_columns:
    stats.probplot(outliers_filtered_df[col], dist="norm", plot=plt)
    plt.title(f'Q-Q Plot de {col}')
    plt.show()

Las transformaciones *Box-Cox* y *Yeo-Johnson* son generalizaciones, pues incluyen casos especiales de transformaciones: logarítmica (`λ = 0`), recíproca (`λ = -1`),  raíz cuadrada (`λ = 0.5`) y más exponentes. En *Box-Cox* se evalúan varios valores de `λ` y se selecciona aquel que arroja la mejor transformación

La transformación de *Yeo-Johnson* es una extensión de la transformación de *Box-Cox* que ya no está restringida a valores positivos. En otras palabras, la transformación de *Yeo-Johnson* se puede utilizar en variables con valores cero y negativos, así como valores positivos. Cuando la variable solo tiene valores positivos, la transformación de *Yeo-Johnson* es como la transformación de *Box-Cox* de la variable más uno.

En este caso, como las variables `price` y `odometer` tienen valores de 0, utilizaremos *Yeo-Johnson*.

Recuerda que en `PowerTransformer()` el parámetro `standardize` por defecto es `True`. Con esto, después de hacer la transformación, el resultado se escala con *Z-score*, haciendo que la media sea 0 y la desviación estándar 1. Como en el ejercicio 6 ocuparás Min-Max para escalar, haz `standardize=False` para omitir el escalamiento en esta fase.

5b) Aplica la transformación Yeo-Johnson para reducir el sesgo en estas variables. Coloca el resultado en un dataframe denominado `transf_df` que contenga sólo las variables transformadas.

In [ ]:
from sklearn.preprocessing import PowerTransformer

transformer = PowerTransformer(method="yeo-johnson", standardize=False)
transformer.fit(outliers_filtered_df[selected_columns])

transformed = transformer.transform(outliers_filtered_df[selected_columns])

transf_df = pd.DataFrame(transformed, columns=selected_columns)


5c) Dibuja nuevamente los gráficos Q-Q y los histogramas para observar el efecto de la transformación.

In [ ]:
# plt.figure(figsize=(15, 6))

# plt.subplot(1, 3, 1)
# stats.probplot(transf_df['price'], dist="norm", plot=plt)
# plt.title(f"Q-Q plot of price")

# plt.subplot(1, 3, 2)
# stats.probplot(transf_df['age'], dist="norm", plot=plt)
# plt.title(f"Q-Q plot of age")

# plt.subplot(1, 3, 3)
# stats.probplot(transf_df['odometer'], dist="norm", plot=plt)
# plt.title(f"Q-Q plot of odometer")


for col in selected_columns:

    stats.probplot(transf_df[col], dist="norm", plot=plt)
    plt.title(f'Q-Q Plot de {col}')
    plt.show()

    sns.histplot(transf_df[col], bins=20, kde=True)
    plt.title(f'Histograma (Transformado) de {col}')
    plt.show()

## Escalando variables numéricas

6a) Como las tres variables poseen diferentes escalas, aplica el método Min-Max (al dataframe obtenido: `transf_df`) para que queden en el rango fijo [0 a 1]. Coloca el resultado en un dataframe denominado `minmax_df`.


In [ ]:
from sklearn import preprocessing

minmax_scale = preprocessing.MinMaxScaler().fit(transf_df[selected_columns])
minmax = minmax_scale.transform(transf_df[selected_columns])

minmax_df = pd.DataFrame(minmax, columns=selected_columns)

6b) Dibuja nuevamente los histogramas para verificar que el escalamiento no cambia la distribución.

In [ ]:
# selected_columns = ['price', 'age', 'odometer']

# Seaborn
fig, axes = plt.subplots(3,1, figsize=(10,15))
plt.subplots_adjust(wspace=0.3)
axes = axes.ravel()

for col, ax in zip(minmax_df[selected_columns], axes):
  sns.histplot(x=minmax_df[col], ax=ax, bins=20, kde = True)
  ax.set(title=f'{col}', xlabel=None)

## Codificando variables categóricas


7a) Utiliza el método `describe()` para obtener el resumen de las estadísticas descriptivas en las variables categóricas.

In [ ]:
outliers_filtered_df.describe(include = "object").T
cars_cat = outliers_filtered_df.select_dtypes(include='object')


7b) Según la cardinalidad observada, separa en dos listas (`highly_cardinal_variables` y `few_cardinal_variables`) los nombres de las variables con alta cardinalidad y baja cardinalidad respectivamente. Una variable es considerada de alta cardinalidad cuando tiene cientos de valores únicos.

In [ ]:
highly_cardinal_variables = []
few_cardinal_variables = []

print(outliers_filtered_df.nunique())
for variable in cars_cat:

    if outliers_filtered_df[variable].nunique() >= 100:
        highly_cardinal_variables.append(variable)
    else:
        few_cardinal_variables.append(variable)

print(f"Highly cardinal variables: {highly_cardinal_variables}")
print(f"Few cardinal variables: {few_cardinal_variables}")

7c) Visualiza en gráficos de barras la frecuencias de las categorías para las variables con baja cardinalidad (en `few_cardinal_variables`)

In [ ]:
fig, axes = plt.subplots(2,4, figsize=(30,10))
plt.subplots_adjust(hspace=0.75)
axes = axes.ravel()
for col, ax in zip(outliers_filtered_df[few_cardinal_variables], axes):
  sns.countplot(x=col, data=outliers_filtered_df, ax=ax)
  ax.set(title=f'{col}', xlabel=None)
  ax.tick_params(axis='x',rotation=90)

8. Aplica codificación *one-hot* a las variables en `few_cardinal_variables`, borrando la primera categoría (`drop='first'`). Coloca el resultado en un dataframe denominado `onehot_df`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# onehot_df = outliers_filtered_df[few_cardinal_variables].copy()

onehot_encoder = OneHotEncoder(drop='first', sparse_output=False)
onehot_ar = onehot_encoder.fit_transform(outliers_filtered_df[few_cardinal_variables])
# The returned ndarray is converted to a dataframe
onehot_df = pd.DataFrame(onehot_ar)
onehot_df.columns = onehot_encoder.get_feature_names_out()
onehot_df


9. Aplica codificación binaria a las variables en `highly_cardinal_variables`. Coloca el resultado en un dataframe denominado `binary_df`.

In [ ]:
# !pip install category_encoders

In [ ]:

from category_encoders.binary import BinaryEncoder

# binary_df = outliers_filtered_df[highly_cardinal_variables].copy()

binary_encoder = BinaryEncoder()
binary_df = binary_encoder.fit_transform(outliers_filtered_df[highly_cardinal_variables])
# Returns directly a dataframe with the names in the columns
binary_df

10. Concatena los dataframes obtenidos: `minmax_df` (cuyas variables numéricas ya están transformadas y escaladas) y las codificaciones anteriores (`onehot_df` y `binary_df`). Toma en cuenta que si los índices de los dataframes no coinciden, la concatenación no se realizará correctamente. Utiliza `reset_index(drop=True, inplace=True)` para restablecer los índices.

In [ ]:
minmax_df.reset_index(drop=True, inplace=True)
onehot_df.reset_index(drop=True, inplace=True)
binary_df.reset_index(drop=True, inplace=True)
final_df = pd.concat([minmax_df, onehot_df, binary_df], axis=1)
final_df